### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/225/Dragon_Ball_GT/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,3):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/225/Dragon_Ball_GT/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  40 non-null     object
dtypes: object(1)
memory usage: 448.0+ bytes
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n4\n\n\nStory\n3\n\n\nAnimatio...
1,\n\n\n\nOverall\n6\n\n\nStory\n4\n\n\nAnimatio...
2,\n\n\n\nOverall\n7\n\n\nStory\n7\n\n\nAnimatio...
3,\n\n\n\nOverall\n4\n\n\nStory\n4\n\n\nAnimatio...
4,\n\n\n\nOverall\n3\n\n\nStory\n3\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

 Overall 4 Story 3 Animation 5 Sound 6 Character 4 Enjoyment 0 Oh boy, I'm stepping into unholy ground here! This review will be going into just about every plot hole, very F up, just about everything that is terrible and wrong in DBGT that I can find or remember. *Just so you know, I am a GIGANTIC DBZ FANBOY, and I do know DBZ isn't "perfect" and yes this review is nothing but a whinny nit-picking troll intended bash fest, so if you hate reading about hate or if you are not familiar with the majority of dragonball and most of the stuff that accurs before GT cause this review will spoil most of the "plot" of GT, then go ahead and click unhelpful * STORY: Sequels, well they usually give justice to the original, or remind you that the original is as good as it gets. Dragonball Z, anime franchise king, the big long winded battle, overrated fail, BEST ANIME EVAR!!, whatever your take on DBZ is, you can not deny it's popularity, calling it a household name in Japan (And around the world) is

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'oh', 'boy', 'stepping', 'into', 'unholy', 'ground', 'here', 'this', 'review', 'will', 'be', 'going', 'into', 'just', 'about', 'every', 'plot', 'hole', 'very', 'up', 'just', 'about', 'everything', 'that', 'is', 'terrible', 'and', 'wrong', 'in', 'dbgt', 'that', 'can', 'find', 'or', 'remember', 'just', 'so', 'you', 'know', 'am', 'gigantic', 'dbz', 'fanboy', 'and', 'do', 'know', 'dbz', 'isn', 'perfect', 'and', 'yes', 'this', 'review', 'is', 'nothing', 'but', 'whinny', 'nit', 'picking', 'troll', 'intended', 'bash', 'fest', 'so', 'if', 'you', 'hate', 'reading', 'about', 'hate', 'or', 'if', 'you', 'are', 'not', 'familiar', 'with', 'the', 'majority', 'of', 'dragonball', 'and', 'most', 'of', 'the', 'stuff', 'that', 'accurs', 'before', 'gt', 'cause', 'this', 'review', 'will', 'spoil', 'most', 'of', 'the', 'plot', 'of', 'gt', 'then', 'go', 'ahead', 'and', 'click', 'unhelpful', 'story', 'sequels', 'well', 'they', 'usually', 'giv

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation_sound', 'character_enjoyment', 'it', 'sucks', 'period', 'it', 'has', 'lots', 'of', 'plot', 'holes', 'and', 'full', 'of', 'contradictions', 'apparently', 'old', 'man', 'krillin', 'is', 'the', 'strongest', 'but', 'lets', 'get', 'to', 'the', 'key', 'points', 'on', 'why', 'it', 'sucks', 'story', 'very', 'poor', 'in', 'the', 'beginning', 'they', 'attempted', 'to', 'recreate', 'the', 'feeling', 'that', 'the', 'original', 'dragonball', 'had', 'but', 'they', 'ultimately', 'failed', 'in', 'later', 'parts', 'of', 'the', 'story', 'they', 'tried', 'to', 'recreate', 'the', 'vegeta', 'saga', 'and', 'later', 'the', 'android', 'saga', 'but', 'they', 'have', 'failed', 'to', 'keep', 'me', 'entertained', 'overall', 'the', 'story', 'was', 'full', 'of', 'disappointment', 'instead', 'of', 'seeing', 'other', 'characters', 'beating', 'the', 'enemies', 'goku', 'ends', 'up', 'doing', 'so', 'which', 'gets', 'kind', 'of', 'repetitive', 'helpful']


In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'character_enjoyment', 'step', 'unholy', 'ground', 'review', 'go', 'plot', 'hole', 'terrible', 'wrong', 'dbgt', 'find', 'remember', 'know', 'gigantic', 'dbz', 'fanboy', 'know', 'dbz', 'perfect', 'review', 'whinny', 'nit', 'pick', 'troll', 'intend', 'bash', 'f', 'hate', 'reading', 'hate', 'familiar', 'majority', 'dragonball', 'stuff', 'accur', 'gt', 'cause', 'review', 'spoil', 'plot', 'gt', 'go', 'ahead', 'click', 'unhelpful', 'story', 'sequel', 'well', 'usually', 'give', 'justice', 'original', 'remind', 'original', 'good', 'get', 'dragonball', 'anime', 'franchise', 'king', 'big', 'long', 'wind', 'battle', 'overrate', 'fail', 'good', 'anime', 'evar', 'take', 'dbz', 'deny', 'popularity', 'call', 'household', 'name', 'japan', 'world', 'understatement', 'probably', 'well', 'good', 'comparison', 'make', 'dbz', 'mario', 'know', 'course', 'thing', 'always', 'happen', 'series', 'become', 'popular', 'sequel', 'make', 'remember', 'find', 'dragonball', 'gt', 'go', 'crazy', 'f

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 2), (1, 9), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 7), (10, 1), (11, 1), (12, 4), (13, 1), (14, 1), (15, 2), (16, 3), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 6), (24, 3), (25, 1), (26, 1), (27, 6), (28, 1), (29, 1), (30, 2), (31, 3), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 5), (38, 2), (39, 3), (40, 1), (41, 4), (42, 2), (43, 2), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 2), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 4), (64, 13), (65, 1), (66, 1), (67, 4), (68, 1), (69, 4), (70, 1), (71, 1), (72, 1), (73, 11), (74, 17), (75, 2), (76, 9), (77, 2), (78, 2), (79, 1), (80, 1), (81, 4), (82, 1), (83, 1), (84, 2), (85, 1), (86, 4), (87, 2), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 1), (105, 3), (106, 1), (107, 1), (108, 1), (109, 2), (110, 

In [21]:
# Using LDA to develop a topic model with 3 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 3 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.020*"gt" + 0.018*"show" + 0.016*"series" + 0.016*"dragon_ball" + 0.011*"even" + 0.011*"say" + 0.011*"watch" + 0.009*"story" + 0.009*"dragon" + 0.009*"great"'), (1, '0.016*"goku" + 0.013*"trunk" + 0.011*"dragonball" + 0.011*"reason" + 0.008*"dbgt" + 0.008*"gt" + 0.008*"come" + 0.008*"series" + 0.008*"people" + 0.008*"go"'), (2, '0.016*"goku" + 0.016*"make" + 0.011*"even" + 0.010*"gt" + 0.009*"series" + 0.008*"well" + 0.007*"ssj" + 0.007*"good" + 0.007*"time" + 0.007*"story"'), (3, '0.016*"character" + 0.016*"story" + 0.014*"overall" + 0.013*"well" + 0.011*"saga" + 0.011*"end" + 0.011*"good" + 0.011*"time" + 0.009*"goku" + 0.009*"watch"'), (4, '0.019*"dragonball" + 0.015*"dbz" + 0.012*"well" + 0.011*"gt" + 0.011*"make" + 0.011*"say" + 0.010*"get" + 0.009*"dragon" + 0.009*"watch" + 0.008*"go"'), (5, '0.016*"story" + 0.015*"character" + 0.013*"make" + 0.012*"dragonball" + 0.011*"goku" + 0.011*"little" + 0.009*"bad" + 0.009*"get" + 0.007*"watch" + 0.006*"overall"'), (6, '0.016*"stor

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.060746 -0.095157       1        1  24.534812
6      0.110503  0.005370       2        1  20.128943
4      0.042020 -0.086401       3        1  16.588242
7      0.049671  0.037313       4        1  14.961277
5     -0.023868  0.074478       5        1  11.742255
3      0.004032  0.023622       6        1   5.601232
0     -0.029530  0.013433       7        1   3.861172
1     -0.092083  0.027341       8        1   2.582068, topic_info=                    Term       Freq      Total Category  logprob  loglift
437          dragon_ball  38.000000  38.000000  Default  30.0000  30.0000
125                   gt  57.000000  57.000000  Default  29.0000  29.0000
74            dragonball  48.000000  48.000000  Default  28.0000  28.0000
262                 show  37.000000  37.000000  Default  27.0000  27.0000
251               series  53.000000  53.000000  Default  26.0000  26.0000
..                   ...        ...        ...      ...      ...      ...
44   character_enjoyment   0.881580  28.651860   Topic8  -5.2304   0.1753
199              overall   0.860454  50.678496   Topic8  -5.2546  -0.4192
43             character   0.868215  65.841389   Topic8  -5.2457  -0.6720
115                  get   0.832802  50.988429   Topic8  -5.2873  -0.4580
121                 good   0.491632  54.278686   Topic8  -5.8144  -1.0476

[580 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1651      6  0.738363  abilitie
1652      6  0.738360   absence
549       1  0.308335  absolute
549       4  0.616670  absolute
758       7  0.790589    absorb
...     ...       ...       ...
329       6  0.092286      year
329       7  0.092286      year
330       1  0.241542       yet
330       3  0.483083       yet
330       8  0.241542       yet

[1003 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 5, 8, 6, 4, 1, 2])

In [23]:
# Save the data frame as a CSV file
# job_datas.to_csv(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Programming Assignment 2\Chan_Matthew_INST447_PA2.csv')

In [24]:
# Week 9 Topic Modelling
# ner_corpus = ' '.join(ner_df['Name'])
# corp = open(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Week 9\corpus.txt',"w")
# corp.writelines(ner_corpus)
# corp.close()